# Heurística QKP $0$-$1$

## Dados

Considere dos seguintes dados:

-  $N = \{1, \ldots, n\}$ um conjunto dos objetos.

-  $P = \{p_{ij} \in \mathbb{R}_{+} : \ i \leq j, \ i, j \in N \}$ o conjunto dos benefícios aos itens de $N$ e a relação entre eles.

-  $W = \{ w_i \in \mathbb{N} : i \in N \}$ o conjunto dos pesos dos itens de $N$.

-  $c \in \mathbb{N}$ capacidade da mochila, onde $\displaystyle\max_{i \in N} w_i \leq c < \sum_{i  \in N} w_i$.

## QKP 0-1

Utilizando os dados acima para definir o problema da mochila quadrática 0-1.

\begin{align}
max \; & \sum_{i \in N} q_{i} x_i + \sum_{i = 1}^{n-1} \sum_{j =i+1}^{n}
p_{ij} x_{i} x_{j} \\
s. a  \; & \sum_{i \in N} w_{i} x_{i} \leq c, \\
& x_{i} \in \left\{0, 1 \right\}, \ \ i  \in N.
\end{align}

onde $q_{i} = p_{ii}, \ i \in N$.

### Um limite inferiore para o QKP

Limites inferiores podem ser obtidos de modo eficiente através de heurísticas. 

Billionnet e Calmels(1996) propuseram uma **heurística** para gerar um limite inferior para o QKP, onde primeiro geram uma solução gulosa inicial definindo $x_{j} = 1$ para todo $j \in N$, e então interativamente trocam o valor das variáveis de $1$ para $0$, de modo a atingir o menor decréscimo na função objetivo, até que uma solução viável é atingida.

Em seguida um procedimento denominado **melhora** é executado, onde uma sequência de iterações são executadas com a finalidade de melhorar a solução através de busca local. 

Seja 
$$S = \left\{ j \in N : \ x_{j} = 1 \right\}$$
o conjunto que define a atual solução.

Para cada $j \in N \setminus S$, se 
$$w_{j} + \displaystyle\sum_{l\in S} w_{l} \leq c$$ 
defina $I_{j} = \emptyset$ e seja $\delta_{j}$ o valor que define o crescimento da função objetivo quando $x_{j}$ toma valor 1, caso contrário, seja $\delta_{j}$ o maior crescimento quando definimos $x_{j} = 1$ e $x_{i} = 0$ para algum $i \in S$ tal que 
$$w_{j} - w_{i} + \displaystyle\sum_{l \in S} w_l \leq c$$ 
e seja $I_{j} = \left\{ i \right\}$

Escolha um $k$ tal que 
$$\delta_{k} = \displaystyle\max_{j \in   N  \setminus S} \delta_{j}$$

A heurística finaliza se $\delta_{k} \leq 0$, caso contrário o corrente conjunto solução é definido como $S \setminus I_{k} \cup \left\{ k \right\}$ e outra iteração é executada.


## Implementação da heuristica

In [19]:
import sys
import numpy as np

In [20]:
def qknapsack(dim,perc,t):

    instance = f"instances/{dim}/{dim}_{perc}_{t}.txt"

    with open(instance, 'r') as file: linhas = file.readlines()

    # remove linha vazia inicial e elimina os "\n" de cada linha
    linhas = [a.strip() for a in linhas] 

    # ler o tamanho da instancia
    n = int(linhas[0]) 

    # ler a diagonal da matriz
    d = np.fromstring(linhas[1], dtype=int, sep = ' ') 

    # define a matriz
    p = np.zeros((n,n), dtype=int) 

    # preenche a diagonal
    for i in range(n): 
        p[i][i] = d[i]

    # preenche o resto da matriz
    for i in range(n-1): 
        linha = np.fromstring(linhas[i+2], dtype=int, sep = ' ')
        for j in range(n-(i+1)):
            p[i][j+i+1] = linha[j]
            p[j+i+1][i] = p[i][j+i+1]

    # ler a capacidade
    c = int(linhas[n+2]) 

    # ler os pesos
    w = np.fromstring(linhas[n+3], dtype=int, sep = ' ') 

    return n, p, w, c

In [21]:
def guloso(n, p, w, c):
    psum = wsum = 0
  
    x = [0 for i in range(n)]
  
    for i in range(n):
        x[i] = 0
        wsum += w[i]
        for j in range(n):
            psum += p[i][j]
  
    ptot = psum
    wtot = wsum
  
    for i in range(n):
        x[i] = 1
  
    while True:
        mineff = ptot
        mini = -1
        for  i in range(n):
            if x[i] == 0:
                continue
            pi = -p[i][i]
            for j in range(n):
                if x[j] != 0:
                    pi += p[j][i] + p[i][j]

            eff = pi / w[i]
            if eff < mineff:
                mineff = eff
                mini = i
                minp = pi
        if mini == -1:
            print("error\n")
            exit
        i = mini
        x[i] = 0
        psum -= minp
        wsum -= w[i]
        if wsum <= c:
            break

    lb = psum
  
    return lb, x

In [22]:
def melhora(n, p, w, c, xprime, lb):

    xstar = [0 for i in range(n)]

    res = c
  
    q = [0 for i in range(n)]
  
    for i in range(n):
        if xprime[i] != 0:
            res -= w[i]
  
    while True:
        for i in range(n):
            tot = p[i][i]
            for j in range(n):
                if (j != i) and (xprime[j] != 0):
                    tot += p[i][j] + p[j][i]
            q[i] = tot
    
        bgain = gaini = gainj = 0
    
        for i in range(n):
            if xprime[i] == 0:
                if w[i] <= res:
                    gain = q[i]
                    if gain > bgain:
                        bgain = gain
                        gaini = i
                        gainj = -1
                else:
                    for j in range(n):
                        if j == i:
                            continue
                        if xprime[j] == 0:
                            continue
            
                        if w[i] - w[j] <= res:
                            gain = q[i] - q[j] - (p[i][j] + p[j][i])
                            if gain > bgain:
                                bgain = gain
                                gaini = i
                                gainj = j
        if bgain == 0:
            break
    
        xprime[gaini] = 1
    
        if gainj != -1:
            xprime[gainj] = 0
            res += w[gainj] - w[gaini]
        else:
            res -= w[gaini]
    
        if res < 0:
            print("error\n")
            exit
                        
    gain = 0
    res = c
    for i in range(n):
        if xprime[i] == 0:
            continue
        res -= w[i]
        for j in range(n):
            if xprime[j] != 0:
                gain += p[i][j]

    if res < 0:
        print("error \n")
        exit 

    if gain > lb:
        lb = gain
        for i in range(n):
            xstar[i] = xprime[i]
    del(q)

    return lb, xstar

In [23]:
# __main__ : “top-level code environment”

# What is the “top-level code environment”?
# __main__ is the name of the environment where top-level code is run. 
# “Top-level code” is the first user-specified Python module that starts running. 
# It’s “top-level” because it imports all other modules that the program needs. 
# Sometimes “top-level code” is called an entry point to the application.

if __name__ == "__main__":

    dim = 10
    perc = 50
    id = 1 
    inst = f"${dim}_${perc}_${id}"

    if len(sys.argv) < 2:
        print("Default data file : " + inst)
    else:
        datafile = sys.argv[1]

    n, p, w, c = qknapsack(dim, perc, id)

    lb, x = guloso(n, p, w, c)
    print("Guloso")
    print("lb = ", lb)
    print("x = ", end="")
    print(x)

    lb, x = melhora(n, p, w, c, x, lb)
    print("Melhora")
    print("lb = ", lb)
    print("x = ", end="")
    print(x)


    #for dim in [100]:
    #    for perc in [50]:
    #        for id in range(1,11):
    #            instance = f"instances/{dim}/{dim}_{perc}_{id}.txt"
    #            qknapsack(dim,perc,id)

Guloso
lb =  695
x = [1, 1, 1, 0, 1, 1, 1, 0, 1, 1]
Melhora
lb =  747
x = [1, 1, 0, 1, 1, 1, 1, 0, 1, 1]
